In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import pdist, cdist
from collections import defaultdict
from scipy import stats
import scipy

## Load the historic player data

In [ ]:
data = json.load(open('real-player.json','rb'))

In [ ]:
df = pd.DataFrame(data['ratings'])

In [ ]:
df = df.drop(['fuzz','abbrev_if_new_row'],1)#.set_index(['slug','season'])

In [ ]:
df = df.set_index(['slug','season']).reset_index()

In [ ]:
cols = list(df.columns[2:])
ovr_weights =  {'diq': 0.093,
 'dnk': 0.0424,
 'drb': 0.0968,
 'endu': 0.0075,
 'fg': -0.0093,
 'ft': 0.049,
 'hgt': 0.225,
 'ins': -0.0143,
 'jmp': 0.0505,
 'oiq': 0.0971,
 'pss': 0.0657,
 'reb': 0.0534,
 'spd': 0.156,
 'stre': 0.0962,
 'tp': 0.105}
ovr_v = np.array([ovr_weights[cols[i]] for i in range(len(cols))])

In [ ]:
ratings  = defaultdict(lambda: dict())
draft_year = defaultdict(lambda:9999999)
final_year = defaultdict(int)
for row in df.itertuples():
    yr = data['bios'][row[1]]['draftYear']
    born = data['bios'][row[1]]['bornYear']
    name = row[1]
    r_yr = row[2]
    if yr is None or yr == 0:
        continue
    if born is None or born ==0:
        continue
    if yr == r_yr:
        continue
    if r_yr >= 2019:
        continue
    if r_yr <= 1983:
        continue
    if yr+1 == r_yr or name in ratings:
        ratings[name][r_yr-born] = list(row[3:])
        draft_year[name] = min(draft_year[name],r_yr)
        final_year[name] = max(final_year[name],r_yr)

In [ ]:
mean_r = []
for a in range(20,40):
    vals = []
    for k,v in ratings.items():
        if a in v and a+1 in v:
            r1 = (np.array(v[a])*ovr_v).sum()
            r2 = (np.array(v[a+1])*ovr_v).sum()
            vals.append([r1,r2])
    vals = np.array(vals)
    mean_r.append(scipy.stats.pearsonr(vals[:,0],vals[:,1])[0])
np.mean(mean_r)

In [ ]:
finish_career = {}
for row in df.groupby('slug').max().itertuples():
    born = data['bios'][row[0]]['bornYear']
    finish_career[row[0]] = row[1]-born in ratings.get(row[0],{})

In [ ]:
ages = np.unique(sum([list(v.keys()) for k,v in ratings.items()],[]))
all_play = sum([[[yr] + v2 for yr, v2 in v.items()] for k,v in ratings.items()],[])
all_play = np.array(all_play)


In [ ]:
youth = np.array([[min(v.keys())] + v[min(v.keys())] for k,v in ratings.items()])
youth = youth[youth[:,0] < 24]
rookie_progs = np.array([[min(v.keys())] + np.array(v[min(v.keys())+1]) - np.array(v[min(v.keys())]) for k,v in ratings.items() if min(v.keys())+1 in v])
rookie_progs = rookie_progs[rookie_progs[:,0]<24]
plt.hist((youth[:,1:] * ovr_v).sum(1),12)

In [ ]:
for a in ages:
    if a > 38:
        continue
    plt.subplot(4,5,a-18)
    r_age = np.array([v[a] for k,v in ratings.items() if a in v])
    plt.imshow(np.cov(r_age,rowvar=False),cmap='RdBu',vmin=-150,vmax=150)
    plt.axis('off')
    #plt.title(str(a))
plt.tight_layout(h_pad=0.1,w_pad=0)

## model classes

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans, MiniBatchKMeans

In [ ]:
class_points = all_play[:,1:].astype(np.float32)
class_points /= ((class_points * ovr_v).sum(1)-6.4)[:,None]

clfk = MiniBatchKMeans(3,n_init=100)
clfk.fit(class_points)

In [ ]:
clfg = GaussianMixture(3,means_init=clfk.cluster_centers_,covariance_type='full')
_ = clfg.fit(class_points)

In [ ]:
class_scale = clfg.means_/class_points.mean(0)
tmp = ['a' for _ in range(3)]
des = ['guard','wing','big']
fix_c = {}
for i2,i in enumerate(np.argsort(class_scale[:,cols.index('hgt')])):
    tmp[i] = des[i2]
    fix_c[i] =i2
c_df = pd.DataFrame(class_scale,columns=cols,index=tmp).round(2)
print(fix_c)
c_df

In [ ]:
types_new = {_[0]:{c:v for c,v in zip(cols,_[1:]) if abs(v-1) > 0.001} for _ in c_df.itertuples()}
types_og = {
"guard": {
		"jmp": 1.65,
		"spd": 1.65,
		"drb": 1.5,
		"pss": 1.5,
		"ft": 1.4,
		"fg": 1.4,
		"tp": 1.4,
		"oiq": 1.2,
		"endu": 1.4,
	},
	"wing": {
		"drb": 1.2,
		"dnk": 1.5,
		"jmp": 1.4,
		"spd": 1.4,
		"ft": 1.2,
		"fg": 1.2,
		"tp": 1.2,
	},
	"big": {
		"stre": 1.2,
		"ins": 1.6,
		"dnk": 1.5,
		"reb": 1.4,
		"ft": 0.8,
		"fg": 0.8,
		"tp": 0.8,
		"diq": 1.2,
	},
}


In [ ]:
clabel = clfg.predict(class_points)
clabel = np.array([fix_c[_] for _ in clabel])
chistbin = np.linspace(0.25,1.75,18)
cmean = all_play[:,1:].mean(0)[6]
c_n = []
chist = []
for i in range(3):
    c_n.append((clabel==i).sum())
    chist.append(np.histogram(class_points[clabel==i,6],chistbin,density=True)[0]+1e-4)
    plt.hist(class_points[clabel==i,6],chistbin,alpha=0.5,density=True,label=str(i))
plt.legend()

In [ ]:
x0_c = [  0.23, -11,  1.2]
x = x0_c
def eval_c(x):
    pred_c = x[0]*all_play[:,7]+x[1]
    np.random.seed(42)
    pred_c = pred_c + x[2]*np.random.randn(all_play.shape[0])
    rnd = np.clip(np.round(pred_c),0,2)
    kl = 0
    c_t = []
    for i in range(3):
        c_t.append((rnd==i).sum())
        phist = np.histogram(all_play[rnd==i,7]/cmean,chistbin,density=True)[0]+1e-4
        kl += (phist * np.log(phist/chist[i])).sum()
        kl += (chist[i] * np.log(chist[i]/phist)).sum()
    if np.isnan(kl):
        return 1e9
    return kl*((np.array(c_t)-np.array(c_n))**2).sum()
print(eval_c(x0_c))

#import cma
#es = cma.CMAEvolutionStrategy(x0_c,0.1)
#res = es.optimize(eval_c)

In [ ]:
#es.mean

In [ ]:
pred_c = x0_c[0]*all_play[:,7]+x0_c[1]
pred_cf = pred_c + x0_c[2]*np.random.randn(all_play.shape[0])
pred_c = np.clip(pred_cf,0,2)
rnd = np.clip(np.round(pred_c),0,2)
plt.scatter(all_play[:,7],pred_cf,c=rnd)

In [ ]:
for i in range(3):
    plt.hist(all_play[rnd==i,7],cmean*chistbin,alpha=0.5,density=True)

## model features

In [ ]:
types_opt={'guard': {'diq': 1.2,
  'dnk': 1.8,
  'drb': 1.5,
  'endu': 1.5,
  'fg': 1.5,
  'ft': 1.3,
  'ins': 1.2,
  'jmp': 1.6,
  'oiq': 1.3,
  'pss': 1.6,
  'spd': 1.3,
  'tp': 1.3},
 'wing': {'diq': 0.9,
  'dnk': 2.1,
  'drb': 1.2,
  'endu': 1.1,
  'fg': 1.3,
  'ft': 1.2,
  'ins': 1.2,
  'jmp': 1.4,
  'oiq': 1.2,
  'pss': 0.8,
  'reb': 1.2,
  'spd': 1.1,
  'tp': 1.2},
 'big': {'dnk': 2.1,
  'drb': 1.2,
  'endu': 1.3,
  'fg': 1.2,
  'ins': 1.4,
  'jmp': 1.3,
  'oiq': 1.2,
  'pss': 1.1,
  'reb': 1.2,
  'spd': 1.1,
  'stre': 1.2}}

In [ ]:
athleticismRatings = ["stre", "spd", "jmp", "endu", "dnk"]
shootingRatings = ["ft", "fg", "tp"]
skillRatings = ["oiq", "diq", "drb", "pss", "reb"]

v1 = np.array([int(_ in athleticismRatings) for _ in cols])
v2 = np.array([int(_ in shootingRatings) for _ in cols])
v3 = np.array([int(_ in skillRatings) for _ in cols])
v4 = np.array([int(_=='ins') for _ in cols])

vmul = np.array([v1,v2,v3,v4])

if False:
    mean_v = np.array([22, 27, 37, 17, 32, 32, 0, 27, 40, 22, 37, 37, 40, 37, 32])
    x0_c = [  0.23, -11,  1.2]
    r_std = [3]
    v_std = [0.2,0.2,0.2,0.2]
    types = types_og
elif False:
    mean_v = np.array([43, 48, 50, 38, 46, 43, 0, 44, 57, 40, 47, 46, 52, 47, 46])
    r_std = [2.9]
    v_std = [0.12,0.17,0.12,0.22]
    types = types_new
elif False:
    mean_v = np.array([42, 43, 38, 28, 36, 36, 0, 40, 36, 37, 37, 46, 36, 49, 38])
    r_std = [3.3]
    v_std = [0.104,0.115,0.104,0.13]
    types = types_og
else:
    mean_v = np.array([42, 43, 38, 29, 36, 35, 0, 40, 36, 37, 37, 45, 36, 49, 38])
    r_std = [3.9]
    v_std = [0.10,0.14,0.11,0.20]
    types = types_og

c_mul = np.array([[types[t].get(c,1) for c in cols] for t in ['guard','wing','big']])


In [ ]:
for c, r in zip(cols,mean_v):
    if c != 'hgt':
        print(c,':',r,',')

In [ ]:
rand_hgt = np.random.randn(12500)*13.6 + 47.5
f_hgt = np.array(list(youth[:,7]) + list(rand_hgt))

simN = f_hgt.shape[0]
pred_c = x0_c[0]*f_hgt+x0_c[1]
pred_cf = pred_c + x0_c[2]*np.random.randn(simN)
pred_c = np.clip(pred_cf,0,2)
rnd = np.clip(np.round(pred_c),0,2).astype(int)

v_m = (((np.random.randn(simN,4) * np.array(v_std) ) @ vmul) + 1)
pred_vec = (mean_v  + r_std[0]*np.random.randn(simN,15))
pred_vec[:,6] = f_hgt
pred_vec = v_m * c_mul[rnd] * pred_vec
pred_vec[:,6] = f_hgt


In [ ]:
_ = plt.hist((youth[:,1:]*ovr_v).sum(1)-6.4,25,alpha=0.5,density=True,label='rpd')
_ = plt.hist((pred_vec*ovr_v).sum(1)-6.4,25,alpha=0.5,density=True,label='gen')
#_ = plt.hist((beta_p2*ovr_v).sum(1)-6.4,20,alpha=0.5,density=True,label='beta')

plt.legend()
print(youth[:,1:].mean(1).std(),pred_vec.mean(1).std())

In [ ]:
plt.subplot(1,2,1)
plt.imshow(np.cov(youth[:,1:],rowvar=False),vmin=-130,vmax=130,cmap='RdBu')
plt.title('real players')
plt.xticks(np.arange(15),cols,rotation=45)
plt.yticks(np.arange(15),cols)
plt.subplot(1,2,2)
plt.imshow(np.cov(pred_vec,rowvar=False),vmin=-130,vmax=130,cmap='RdBu')
plt.title('generated')
plt.xticks(np.arange(15),cols,rotation=45)
_ = plt.yticks(np.arange(15),cols)

In [ ]:
PC = 50
s1 = (youth[:,1:]*ovr_v).sum(1)
s1 = s1 > np.percentile(s1,PC)
s2 = (pred_vec*ovr_v).sum(1)
s2 = s2 > np.percentile(s2,PC)
plt.subplot(1,2,1)
plt.imshow(np.cov(youth[s1,1:],rowvar=False),vmin=-130,vmax=130,cmap='RdBu')
plt.title('real players')
plt.xticks(np.arange(15),cols,rotation=45)
plt.yticks(np.arange(15),cols)
plt.subplot(1,2,2)
plt.imshow(np.cov(pred_vec[s2],rowvar=False),vmin=-130,vmax=130,cmap='RdBu')
plt.title('generated')
plt.xticks(np.arange(15),cols,rotation=45)
_ = plt.yticks(np.arange(15),cols)

In [ ]:
og_mean_v = mean_v 
og_r_std = r_std
og_v_std = v_std

In [ ]:
def eval_f(params):
    #np.random.seed(43)
    mean_v = np.exp(params[:15])
    r_std = np.exp(params[15:16])
    v_std = np.exp(params[16:20])
    cmul2 = c_mul#np.exp(params[20:]).reshape((3,15))
    res = []
    print(mean_v,r_std,v_std,cmul2)
    for i in range(30):
        np.random.seed(542+i)
        
        rand_hgt = np.random.randn(1500)*13.6 + 47.5
        f_hgt = np.array(list(youth[:,7]) + list(rand_hgt))

        simN = f_hgt.shape[0]
        pred_c = x0_c[0]*f_hgt+x0_c[1]
        pred_cf = pred_c + x0_c[2]*np.random.randn(simN)
        pred_c = np.clip(pred_cf,0,2)
        rnd = np.clip(np.round(pred_c),0,2).astype(int)

        v_m = (((np.random.randn(simN,4) * np.array(v_std) ) @ vmul) + 1)
        pred_vec = (mean_v  + r_std[0]*np.random.randn(simN,15))
        pred_vec[:,6] = f_hgt
        pred_vec = v_m * cmul2[rnd] * pred_vec
        pred_vec[:,6] = f_hgt

        # filter to only the top half with good stats
        s1 = (youth[:,1:]*ovr_v).sum(1)
        s1 = s1 > np.percentile(s1,50)
        s2 = (pred_vec*ovr_v).sum(1)
        s2 = s2 > np.percentile(s2,50)
        
        cov_err = ((np.cov(youth[s1,1:],rowvar=False)-np.cov(pred_vec[s2],rowvar=False))**2).mean()
        cov_err2 = ((np.cov(youth[:,1:],rowvar=False)-np.cov(pred_vec,rowvar=False))**2).mean()

        v1 = (ovr_v*youth[s1,1:]).sum(1)
        v2 = (ovr_v*pred_vec[s2]).sum(1)
        
        tb = min(100,int(np.ceil(max(v1.max(),v2.max()))))
        bb = max(1,int(np.floor(min(v1.min(),v2.min()))))
        sN = tb-bb
        hist_set = np.linspace(bb,tb,sN)
        h1 = np.histogram(v1,hist_set,density=True)[0]+1e-4
        h2 = np.histogram(v2,hist_set,density=True)[0]+1e-4
        kl1 = (np.log(h1/h2)*h1).sum()
        kl2 = (np.log(h2/h1)*h2).sum()
        mean_err = kl1+kl2
        
  
        mean_err2 = ((youth[s1,1:].mean(0)-pred_vec[s2].mean(0))**2).sum()
        mean_err3 = ((youth[:,1:].mean(0)-pred_vec.mean(0))**2).sum()
        std_err1 = ((youth[s1,1:].std(0)-pred_vec[s2].std(0))**2).sum()
        std_err2 = ((youth[:,1:].std(0)-pred_vec.std(0))**2).sum()
        
        youth_pred_pos =  x0_c[0]*youth[:,7]+x0_c[1]+ x0_c[2]*np.random.randn(youth.shape[0])
        youth_pred_pos = np.clip(np.round(youth_pred_pos),0,2).astype(int)
        errs_pos = []
        for i in range(3):
            m1 = youth[s1 & (youth_pred_pos==i),1:]
            m2 = pred_vec[s2 & (rnd==i)]
            errs_pos.append(np.linalg.norm(m1.mean(0)-m2.mean(0)))
            errs_pos.append(np.linalg.norm(m1.std(0)-m2.std(0)))
            errs_pos.append(((np.cov(m1,rowvar=False)-np.cov(m2,rowvar=False))**2).mean())
        
        err_pos = np.prod(errs_pos)#**(1/len(errs_pos))
        
        res.append( err_pos* std_err1*(std_err2**0.1)*cov_err*mean_err*mean_err2*(mean_err3**0.1)*(cov_err2**0.1 ) )
        if np.isnan(res[-1]):
            return 1e20
    #params_err = np.sqrt((np.exp(params[20:])-1)**2).sum()
    return ( np.mean(sorted(res)) ) ** (1/5.3) # 11.3 ?
x0 = list(mean_v+1e-2) + r_std + v_std

#x0 = np.hstack([x0,c_mul.ravel()])

eval_f(np.log(x0))

In [ ]:
import cma
es = cma.CMAEvolutionStrategy(np.log(x0),0.001)
#es.optimize(eval_f)

In [ ]:
#np.exp(es.mean)

In [ ]:
#types_opt = {}
#for c,vec in zip(["guard","wing","big"],np.exp(es.best.x)[20:].reshape((3,15))):
#    types_opt[c] = {c2:round(r,1) for c2,r in zip(cols,vec) if round(r,1) != 1.0 and c2 != 'hgt'}
#types_opt

## Model development

In [ ]:
import scipy
ratings2  = {}
for row in df.itertuples():
    ratings2[(row[1],row[2])] = list(row[3:])
    

# only use recent-ish players
from collections import defaultdict
player_year_rate = defaultdict(dict)
for i,r in ratings2.items():
    if data['bios'][i[0]]['bornYear'] < 1956:
        continue
    if i[1] >= 2019:
        continue
    age=  i[1]-data['bios'][i[0]]['bornYear']
    player_year_rate[i[0]][age] = np.array(r)
SMOOTHING_STD = 0.9
play_year_rateSmooth = {}
for key,play in player_year_rate.items():
    minY = min(play.keys())
    maxY = max(play.keys())
    res = []
    for i in range(minY,maxY+1):
        #res.append(play.get(i,[np.nan for j in range(15)]))
        res.append(play[i] if i in play else res[-1])
    res = np.array(res).astype(float)
    reS = scipy.ndimage.gaussian_filter1d(res,SMOOTHING_STD,mode='nearest',axis=0,truncate=10)
    p2 = {}
    for idx,age in enumerate(range(minY,maxY+1)):
        if age in play:
            p2[age] = reS[idx]
    play_year_rateSmooth[key] = p2

In [ ]:
TRANS_FUNC = lambda x: x#np.sqrt(x)#np.log(x+1e-6)#x**(1/2)#np.sqrt(x)
INV_FUNC = lambda x: x#x**2#np.exp(x)#x**2#x**2
r1 = []
r2 = []
for play in play_year_rateSmooth.values():
    for age,r in play.items():
        if age-1 in play:
            age2 = age-1
            if age2 > 36:
                continue
            r1.append(TRANS_FUNC(play[age]) -TRANS_FUNC(play[age-1]))
            r2.append(age2)
r1 = np.array(r1)
r2 = np.array(r2)


In [ ]:
age_res = []
for age in sorted(np.unique(r2)):
    age_res.append(r1[r2==age].mean(0))
age_res = np.array(age_res)
for i in range(15):
    plt.plot(sorted(np.unique(r2)),age_res[:,i],label=cols[i],c=plt.cm.tab20(i))
plt.xlim(right=35)
plt.legend()
#plt.ylim(-0.2,0.2)

In [ ]:
import sklearn.linear_model as linear_model

TIMES_TO_FIT = 1

clf_models = []
for i in range(TIMES_TO_FIT):
    clf = linear_model.RidgeCV(np.logspace(-5,5,11),cv=5)#SGDRegressor('epsilon_insensitive',alpha=1e-5,epsilon=0,max_iter=10000,tol=1e-9,eta0=1e-5)
    clf.fit(np.repeat(r2,15)[:,None],r1.ravel())
    score = clf.score(np.repeat(r2,15)[:,None],r1.ravel())
    clf_models.append((score,i,clf))
best_model = sorted(clf_models)[-1]
clf = best_model[2]
print(best_model[0])
main_model = (clf.coef_[0] , clf.intercept_) # 0.0855008819536307

In [ ]:
plt.plot(np.unique(r2),np.unique(r2)*main_model[0] +main_model[1])
plt.plot([19,35],[0,0],c='k',ls='--')
plt.grid(True)

In [ ]:
models = []
for i in range(r1.shape[1]):
    clf_models = []
    for j in range(TIMES_TO_FIT):
        clf = linear_model.RidgeCV(np.logspace(-5,5,11),cv=5)#SGDRegressor('epsilon_insensitive',alpha=1e-5,epsilon=0,max_iter=10000,tol=1e-9,eta0=1e-5)
        clf.fit(np.array(r2)[:,None],r1[:,i]-(main_model[0]*r2+main_model[1]))
        score = clf.score(np.array(r2)[:,None],r1[:,i]-(main_model[0]*r2+main_model[1]))
        clf_models.append((score,j,clf))
    best_model = sorted(clf_models)[-1]
    clf = best_model[2]
    print(cols[i],best_model[0])
    models.append((clf.coef_[0],clf.intercept_))

In [ ]:
plt.style.use('seaborn-white')
for i in range(r1.shape[1]):
    plt.plot(np.unique(r2),np.unique(r2)*models[i][0]+models[i][1],label=cols[i],c=plt.cm.tab20(i))
plt.legend()
#plt.xlim(19,34)
#plt.ylim(-4,4)
plt.grid(True)

In [ ]:
means_expected = []
for i in range(r1.shape[1]):
    means_expected.append((models[i][0]*r2 + models[i][1]) * (main_model[0]*r2+main_model[1]) )

In [ ]:
# rank1 approximations of this would be really cool
# but sampling multivariate Gaussians seems... annoying?
removed_means = r1 - np.array(means_expected).T

In [ ]:
age_w = []
ages = sorted(np.unique(r2))
age_stds = []
for age in ages:
    age_w.append((r2==age).sum())
    age_stds.append(removed_means[r2==age].std(axis=0))
age_stds = np.array(age_stds)
age_w = np.array(age_w)
age_w = age_w/age_w.mean()

In [ ]:
base_model = list(main_model) + [age_stds.mean()]

In [ ]:
std_models = []
for i in range(15):
    std_models.append((age_stds[:,i]-base_model[2]).std())

In [ ]:
plt.style.use('seaborn-white')
for i in range(r1.shape[1]):
    plt.plot(np.unique(r2),np.ones_like(np.unique(r2))*std_models[i],label=cols[i],c=plt.cm.tab20(i),lw=3)
plt.legend()
plt.xlim(19,34)
plt.grid(True)

In [ ]:
models

In [ ]:
dat_print = {cols[i]:tuple(np.round(row,4)) for i,row in enumerate(np.hstack([models,np.array(std_models)[:,None]]))}

In [ ]:
print('{} {},'.format("base",list(np.round(base_model,4))))
for k,v in dat_print.items():
    if k == 'hgt':continue
    print('{}: {},'.format(k,list(v)))

In [ ]:
np.quantile(means_expected,0.99,axis=0).mean(),np.quantile(means_expected,0.01,axis=0).mean()

In [ ]:
np.quantile(r1,0.99,axis=0).mean(),np.quantile(r1,0.01,axis=0).mean()

## Jointly do it

In [ ]:
retire_year = defaultdict(list)
for k,v in ratings.items():
    #if not finish_career[k]:
    #    continue
    for idx,i in enumerate(sorted(v)):
        retire_year[idx].append(i+1 in v)
retire_year = {k:np.mean(v) for k,v in retire_year.items()}
plt.plot(list(retire_year.keys()),list(retire_year.values()))

In [ ]:
draft_year['jamesle01']

In [ ]:
sages_real = []
res_real = []
year_league = []
vec_real = []

sages_real_out = []
vec_real_out = []

for i in range(20):
    
    tmp = [v[min(v)+i] for k,v in ratings.items() if len(v) > 0 and min(v)+i in v and draft_year[k] < 2003]
    tmps = [min(v)+i for k,v in ratings.items() if len(v) > 0 and min(v)+i in v and draft_year[k] < 2003]

    res_real.append(np.array(tmp*ovr_v).sum(1))
    vec_real.append(tmp)

    sages_real.append(tmps)
    year_league.append(np.ones_like(res_real[-1])*i)
    
    tmp_out = [v[min(v)+i] for k,v in ratings.items() if len(v) > 0 and min(v)+i in v and min(v)+i+1 not in v and draft_year[k] < 2003]
    tmps_out = [min(v)+i for k,v in ratings.items() if len(v) > 0 and min(v)+i in v and min(v)+i+1 not in v and draft_year[k] < 2003]

    vec_real_out.append(tmp_out)

    sages_real_out.append(tmps_out)
    
sages_real = np.array(sum(sages_real,[]))
res_real = np.array(sum([list(_) for _ in res_real],[]))
vec_real = np.array(sum([list(_) for _ in vec_real],[]))

sages_real_out = np.array(sum(sages_real_out,[]))
vec_real_out = np.array(sum([list(_) for _ in vec_real_out],[]))

year_league = np.array(sum([list(_) for _ in year_league],[]))

print(sages_real.shape[0],sages_real_out.shape,vec_real_out.shape)

In [ ]:
progs_vec = np.vstack([base_model,np.hstack([models,np.array(std_models)[:,None]])])

In [ ]:
class1 = np.vstack([(sages_real),(vec_real*ovr_v).sum(1)]).T
class2 = np.vstack([(sages_real_out),(vec_real_out*ovr_v).sum(1)]).T
rX = np.vstack([class1,class2])
rY = np.hstack([np.ones(class1.shape[0]),np.zeros(class2.shape[0])])
import sklearn.linear_model as linear

r_clf = linear.LogisticRegressionCV(cv=3,Cs=100,scoring='neg_log_loss',class_weight='balanced')
r_clf.fit(rX,rY)
r_clf.coef_,r_clf.intercept_,r_clf.score(rX,rY)

In [ ]:
plt.hist((vec_real*ovr_v).sum(1),density=True,alpha=0.5,label='stay')
plt.hist((vec_real_out*ovr_v).sum(1),density=True,alpha=0.5,label='retire')
plt.legend()
(vec_real*ovr_v).sum(1).mean(),(vec_real_out*ovr_v).sum(1).mean()

In [ ]:
pos_team_coach = np.array(list(0.75+0.25*np.arange(0,15)/15) + list(1+0.25*np.arange(1,16)/15))
neg_team_coach = pos_team_coach[::-1]
limits = [-1,4,-3,5]

x0_f = list(np.log(list(og_mean_v+1e-2) + list(og_r_std)+ list(og_v_std))) + list(progs_vec.ravel()) + list(limits)
x0_f = np.array(x0_f)
x0_og = x0_f
params = x0_f
progs_vec.shape

In [ ]:
youth.shape

In [ ]:
def generate_players(mean_v,r_std,v_std,simN=1500):
    f_hgt = np.random.randn(simN)*13.6 + 47.5
    # kind of sloppy but want the distribution of youth too?
    #_hgt = np.array(list(youth[:,7]) + list(rand_hgt))

    pred_c = x0_c[0]*f_hgt+x0_c[1]
    pred_cf = pred_c + x0_c[2]*np.random.randn(simN)
    pred_c = np.clip(pred_cf,0,2)
    rnd = np.clip(np.round(pred_c),0,2).astype(int)

    v_m = (((np.random.randn(simN,4) * np.array(v_std) ) @ vmul) + 1)
    pred_vec = (mean_v  + r_std[0]*np.random.randn(simN,15))
    pred_vec[:,6] = f_hgt
    pred_vec = v_m * c_mul[rnd] * pred_vec
    pred_vec[:,6] = f_hgt
    
    return pred_vec, rnd

def eval_draft(set1,set2,set2_class):
    s1 = (set1*ovr_v).sum(1)
    s1 = s1 > np.percentile(s1,50)
    s2 = (set2*ovr_v).sum(1)
    s2 = s2 > np.percentile(s2,50)

    cov_err = ((np.cov(set1[s1],rowvar=False)-np.cov(set2[s2],rowvar=False))**2).mean()
    cov_err2 = ((np.cov(set1,rowvar=False)-np.cov(set2,rowvar=False))**2).mean()

    v1 = (ovr_v*set1[s1]).sum(1)
    v2 = (ovr_v*set2[s2]).sum(1)

    tb = min(100,int(np.ceil(max(v1.max(),v2.max()))))
    bb = max(1,int(np.floor(min(v1.min(),v2.min()))))
    sN = tb-bb+1
    hist_set = np.linspace(bb,tb,sN)

    h1 = np.histogram(v1,hist_set,density=True)[0]+1e-4
    h2 = np.histogram(v2,hist_set,density=True)[0]+1e-4
    kl1 = (np.log(h1/h2)*h1).sum()
    kl2 = (np.log(h2/h1)*h2).sum()
    mean_err = kl1+kl2


    mean_err2 = ((set1[s1].mean(0)-set2[s2].mean(0))**2).sum()
    mean_err3 = ((set1.mean(0)-set2.mean(0))**2).sum()
    std_err1 = ((set1[s1].std(0)-set2[s2].std(0))**2).sum()
    std_err2 = ((set1.std(0)-set2.std(0))**2).sum()

    youth_pred_pos =  x0_c[0]*set1[:,6]+x0_c[1]+ x0_c[2]*np.random.randn(set1.shape[0])
    youth_pred_pos = np.clip(np.round(youth_pred_pos),0,2).astype(int)
    errs_pos = []
    for i in range(3):
        m1 = set1[s1 & (youth_pred_pos==i)]
        m2 = set2[s2 & (set2_class==i)]
        errs_pos.append(np.linalg.norm(m1.mean(0)-m2.mean(0)))
        errs_pos.append(np.linalg.norm(m1.std(0)-m2.std(0)))
        errs_pos.append(((np.cov(m1,rowvar=False)-np.cov(m2,rowvar=False))**2).mean())

    err_pos = np.prod(errs_pos)#**(1/len(errs_pos))

    err_pos = err_pos* std_err1*(std_err2**0.1)*cov_err*mean_err*mean_err2*(mean_err3**0.1)*(cov_err2**0.1 )
    gen_err = ( err_pos) ** (1/5.3)
    return gen_err
gen_p, rnd_p = generate_players(mean_v,r_std,v_std)
eval_draft(youth[:,1:],gen_p,rnd_p)

In [ ]:
def progress_players(prog_model, prog_limits, initial_vec):
    simN = initial_vec.shape[0]
    sages = []
    res_vec = []
    y2y_corr = []
    y2y_weight = []
    work_rate = np.round(np.clip(initial_vec,1,100))
    sim_age = np.random.choice(range(19,23),size=simN,p=[0.5,0.25,0.125,0.125])
    valid_players = np.ones(simN).astype(bool)
    sages_out = []
    res_vec_out = [] 
    coach_sel = np.random.randint(0,30,simN)

    for i in range(25):
        progs_mean = sim_age[:,None]*prog_model[:,0]+prog_model[:,1]
        progs_noise = np.random.randn(simN,16)*prog_model[:,2]
        bc_prog = progs_mean[:,0] + np.clip(progs_noise[:,0],prog_limits[0],prog_limits[1])
        bc_prog = bc_prog*np.where(bc_prog >0,pos_team_coach[coach_sel],neg_team_coach[coach_sel])
        base_change = TRANS_FUNC(work_rate)+ bc_prog[:,None]
        final_change = INV_FUNC(base_change + progs_mean[:,1:] + np.clip(progs_noise[:,1:],prog_limits[2],prog_limits[3]))
        work_rate_new = np.round(np.clip(final_change,1,100))
        #print(work_rate.shape,work_rate_new.shape,valid_players.sum())
        r_corr = scipy.stats.pearsonr((work_rate[valid_players]*ovr_v).sum(1),(work_rate_new[valid_players]*ovr_v).sum(1))[0]
        y2y_corr.append(r_corr)
        y2y_weight.append(valid_players.sum())
        work_rate_old = work_rate.copy()
        work_rate = work_rate_new
        #p_ovrs = (work_rate[valid_players]*ovr_v).sum(1)
        p_ovrs_full =  (work_rate_old*ovr_v).sum(1)#+ 3*np.random.randn(simN)

        
        rXp = np.vstack([sim_age,p_ovrs_full]).T
        valid_vec =  r_clf.predict_proba(rXp)[:,1] > np.random.rand(simN)
        #print(i,(valid_vec &valid_players).sum()/valid_players.sum())
        #print(scipy.stats.pearsonr(valid_vec,p_ovrs_full)[0])
        #retire_ovr = np.percentile(p_ovrs,100-retire_year[i]*100)
        #valid_vec =  p_ovrs_full >= retire_ovr

        sages.append(sim_age[valid_players]+1)
        res_vec.append(work_rate[valid_players])
        sages_out.append(sim_age[valid_players & (~valid_vec)])
        res_vec_out.append(work_rate[valid_players & (~valid_vec)])
        
        # move on
        sim_age += 1
        valid_players = valid_players & valid_vec
        
        if valid_players.sum() < 2:
            break
    sages = np.array(sum([list(_) for _ in sages],[]))
    res_vec = np.array(sum([list(_) for _ in res_vec],[]))
    sages_out = np.array(sum([list(_) for _ in sages_out],[]))
    res_vec_out = np.array(sum([list(_) for _ in res_vec_out],[]))
    
    y2y_corr = np.array(y2y_corr)
    y2y_weight = np.array(y2y_weight)
    
    return sages,res_vec,sages_out,res_vec_out, y2y_corr, y2y_weight
sim_age, sim_rating, sim_out_age, sim_out_rating, y2y_c, y2y_w = progress_players(progs_vec,limits,gen_p)
print(sim_age.shape,sim_out_age.shape)

In [ ]:
plt.hist((vec_real*ovr_v).sum(1),25,alpha=0.5,density=True)
plt.hist((sim_rating*ovr_v).sum(1),25,alpha=0.5,density=True)

In [ ]:
def eval_traj_err(s_ages,s_ratings,p_ages,p_ratings):
    weights = []
    bin_edges = np.array([-100] + list(np.linspace(53,75,23)) + [200])
    traj_errs = []
    for a in range(20,40):
        real_filt = s_ages==a
        sim_filt = p_ages==a
        wt = real_filt.sum()
        if real_filt.sum() < 2 or sim_filt.sum() < 2:
            continue

        #print(real_filt.sum(),sim_filt.sum(),s_ratings.shape,p_ratings.shape,real_filt.shape,sim_filt.shape)
        realV = s_ratings[real_filt]
        simV = p_ratings[sim_filt]
        weights.append(wt)

        real_covar = np.cov(realV,rowvar=False)
        sim_covar = np.cov(simV,rowvar=False)
        covar_err = np.linalg.norm(real_covar.ravel() - sim_covar.ravel())

        real_ovr = (realV*ovr_v).sum(1)
        sim_ovr = (simV*ovr_v).sum(1)
        real_hist = np.histogram(real_ovr,bin_edges,density=True)[0]+1e-9
        sim_hist = np.histogram(sim_ovr,bin_edges,density=True)[0]+1e-9
        kl1 = (real_hist * np.log(real_hist/sim_hist)).sum()
        kl2 = (sim_hist * np.log(sim_hist/real_hist)).sum()
        ovr_err = kl1+kl2

        mean_err = np.linalg.norm(realV.mean(0)-simV.mean(0))
        std_err = np.linalg.norm(realV.std(0)-simV.std(0))

        traj_errs.append([covar_err,ovr_err,mean_err,std_err])
    weights = np.array(weights)
    weights = weights/weights.sum()
    traj_err = np.prod((np.array(traj_errs) *weights[:,None]).sum(0))
    return traj_err
t_err1 = eval_traj_err(sages_real,vec_real,sim_age,sim_rating)
t_err2 = eval_traj_err(sages_real_out,vec_real_out,sim_out_age,sim_out_rating)
t_err1,t_err2


In [ ]:
def eval_system(params):
    mean_v = np.exp(params[:15])
    r_std = np.exp(params[15:16])
    v_std = np.exp(params[16:20])
    p_model = np.array(params[20:68]).reshape((16,3))
    n_limits = params[68:72]
    p_model[7,:] *= 0 # no hgt model
    
    total_errors = []
    for i in range(10):
        np.random.seed(9411+i)
        
        gen_p, rnd_p = generate_players(mean_v,r_std,v_std)
        gen_err = eval_draft(youth[:,1:],gen_p,rnd_p)
        
        sim_age, sim_rating, sim_out_age, sim_out_rating, y2y_c, y2y_w = progress_players(p_model,n_limits,gen_p)

        y2y_w = np.array(y2y_w)
        y2y_w = y2y_w/y2y_w.sum()
        r2r_err = (abs(np.array(y2y_c)-0.826)*y2y_w).sum()

        t_err1 = eval_traj_err(sages_real,vec_real,sim_age,sim_rating)
        t_err2 = eval_traj_err(sages_real_out,vec_real_out,sim_out_age,sim_out_rating)
        
        total_errors.append(  (t_err1*(t_err2**0.4)*(r2r_err**0.1)*(gen_err**0.2))**(1/1.7) )
    final_error = np.mean(total_errors)
    if np.isnan(final_error):
        return 1e7
    return final_error

x0_1 = np.array([ 3.67373654e+00,  3.66905625e+00,  3.68390410e+00,  3.13469021e+00,
        3.52026399e+00,  3.57726975e+00,  1.72220416e+01,  3.62462646e+00,
        3.54796892e+00,  3.62621522e+00,  3.65832143e+00,  3.81477982e+00,
        3.61031017e+00,  3.83607014e+00,  3.69402878e+00, -1.88580283e-01,
       -1.05560750e+01, -2.17844662e+00, -2.53289570e+00, -3.44600098e+00,
       -3.26663088e-01,  6.24595471e+00,  4.20215185e+00, -9.98221236e-02,
        2.84101693e+00, -9.49736583e-01, -5.20868701e-02,  1.78076764e+00,
        1.20502905e+00,  9.65965624e-02, -3.05958759e+00, -1.44558919e-02,
       -5.20189668e-01,  1.38421204e+01,  2.30124850e+00,  1.45674125e-02,
        6.95388511e-02,  5.44261072e-01,  1.54549623e-01, -3.89062486e+00,
       -7.06830317e-02, -2.26848916e-01, -1.53908669e+01, -4.51786142e-01,
       -3.20204505e-02,  9.24024797e-01,  7.56065408e-01, -2.46864504e-01,
        5.44631006e+00,  1.48628803e+00,  7.64093838e-02, -2.03944229e+00,
        4.05732637e-01,  1.57065309e-01, -4.60172584e+00,  2.87683167e-01,
        4.24367515e-02, -9.64028846e-01, -9.81072076e-02, -5.73373608e-02,
        4.40467853e-01,  3.23263336e-01, -9.89722896e-02,  2.67530918e+00,
        2.30973584e-01,  1.37678614e-01, -3.90892064e+00,  6.79816488e-01,
       -1.01514319e+00,  4.08077471e+00, -3.00145527e+00,  4.90089870e+00])

x0_12 = np.array([ 3.72257455,  3.72976193,  3.71423065,  3.27672969,  3.4913225 ,
        3.48569829, -3.32094681,  3.60413194,  3.56845201,  3.57529994,
        3.64165549,  3.80845703,  3.67881184,  3.89516036,  3.7648139 ,
        1.26562861, -4.15848959, -2.09103794, -3.30725258, -1.97664841,
       -0.30662487,  6.13758246,  3.00124879,  0.1250035 , -2.64406875,
        0.23382601,  0.08818039, -2.03138477,  0.18569866,  0.10443687,
       -3.40891316,  0.11259687, -0.34268688,  9.12871378,  0.64275184,
        0.01154811,  0.01841008,  0.14835288,  0.05987082, -1.20433936,
        0.15866409,  0.16905976, -3.57995197,  0.34529105,  0.04619042,
       -1.0178527 ,  0.36117147, -0.16308519,  2.88359828,  0.87373374,
       -0.05358755,  1.49363197,  0.19903506,  0.08685232, -2.5975197 ,
        0.10171959,  0.09052306, -2.0781758 ,  0.22330404, -0.14960772,
        2.08126933,  0.36392916, -0.03694724,  0.85825729,  0.20325397,
        0.07512503, -3.19677288,  0.41691565, -1.08579965,  5.55665434,
       -3.65860969,  5.80256083])
x0_f = x0_og
eval_system(x0_f) # 21.8 linear


In [ ]:
es_final = cma.CMAEvolutionStrategy(x0_f,1,{'CMA_stds':[abs(_)/10 for _ in x0_f],'popsize':45}) # 
es_final.optimize(eval_system)

In [ ]:

if False:
    from cmaes import CMA

    optimizer = CMA(mean=x0_f, sigma=0.01,population_size=16)

    for generation in range(5000):
        solutions = []
        for _ in range(optimizer.population_size):
            x = optimizer.ask()
            value = eval_system(x)
            solutions.append((x, value))
            #print(f"#{generation} {value}")
        v = np.array([_[1] for _ in solutions])
        print('f#{} {:.2f} {:.2f} {:.2f}'.format(generation,v.mean(),v.min(),v.std()))
        optimizer.tell(solutions)

In [ ]:
#configs = [optimizer.ask() for _ in range(100)]
#sorted([(eval_system(_),_) for _ in configs])[0][1]

In [ ]:
es_final.best.x

In [ ]:
params = x0_f
mean_v2 = np.exp(params[:15])
r_std2 = np.exp(params[15:16])
v_std2 = np.exp(params[16:20])
p_model2 = np.array(params[20:68]).reshape((16,3))
n_limits2 = params[68:72]
p_model2[7,:] *= 0

gen_p, rnd_p = generate_players(mean_v2,r_std2,v_std2)
gen_err = eval_draft(youth[:,1:],gen_p,rnd_p)

sim_age, sim_rating, sim_out_age, sim_out_rating, y2y_c, y2y_w = progress_players(p_model2,n_limits2,gen_p)

y2y_w = np.array(y2y_w)
y2y_w = y2y_w/y2y_w.sum()
r2r_err = (abs(np.array(y2y_c)-0.826)*y2y_w).sum()

t_err1 = eval_traj_err(sages_real,vec_real,sim_age,sim_rating)
t_err2 = eval_traj_err(sages_real_out,vec_real_out,sim_out_age,sim_out_rating)

t_err_t = (t_err1*t_err2*r2r_err*(gen_err**0.1))**(1/3.1) 
t_err_t

In [ ]:
#retire_year

In [ ]:
t_err1,t_err2

In [ ]:
t_err1,t_err2,r2r_err,gen_err
# (6166.472371806296, 0.10848798097683329, 6552.872338463377), 580, 0.3, 6.3, 6.9
# (36739.73623341423, 0.1038815935569617, 6155.047251477428), 560, 0.6, 19.4, 6.6
#(6093.079138098562, 56403.49254850698, 0.04840360363522021, 33063.55880033413)

In [ ]:

xp = sorted(np.unique(sages_real))
yp = []
yp_s = []
for a in xp:
    r = (vec_real[sages_real==a]*ovr_v).sum(1)
    yp.append(r.mean())
    yp_s.append(r.std())
yp = np.array(yp)
yp_s = np.array(yp_s)

xp2 = sorted(np.unique(sages_real_out))
yp2 = []
yp_s2 = []
for a in xp2:
    r = (vec_real_out[sages_real_out==a]*ovr_v).sum(1)
    yp2.append(r.mean())
    yp_s2.append(r.std())
yp2 = np.array(yp2)
yp_s2 = np.array(yp_s2)

plt.style.use('fivethirtyeight')
plt.plot(xp,yp,label='remain')
plt.fill_between(xp,yp+yp_s,yp-yp_s,alpha=0.5)
plt.plot(xp2,yp2,label='retire')
plt.fill_between(xp2,yp2+yp_s2,yp2-yp_s2,alpha=0.5)

plt.xlabel('age')
plt.ylabel('ovr')
plt.grid(True)
plt.xlim(20,37)
plt.ylim(45,65)
plt.legend(frameon=True)

In [ ]:

xp = sorted(np.unique(sim_age))
yp = []
yp_s = []
for a in xp:
    r = (sim_rating[sim_age==a]*ovr_v).sum(1)
    yp.append(r.mean())
    yp_s.append(r.std())
yp = np.array(yp)
yp_s = np.array(yp_s)

xp2 = sorted(np.unique(sim_out_age))
yp2 = []
yp_s2 = []
for a in xp2:
    r = (sim_out_rating[sim_out_age==a]*ovr_v).sum(1)
    yp2.append(r.mean())
    yp_s2.append(r.std())
yp2 = np.array(yp2)
yp_s2 = np.array(yp_s2)

plt.style.use('fivethirtyeight')
plt.plot(xp,yp,label='remain')
plt.fill_between(xp,yp+yp_s,yp-yp_s,alpha=0.5)
plt.plot(xp2,yp2,label='retire')
plt.fill_between(xp2,yp2+yp_s2,yp2-yp_s2,alpha=0.5)

plt.xlabel('age')
plt.ylabel('ovr')
plt.grid(True)
plt.xlim(20,37)
plt.ylim(45,75)
plt.legend(frameon=True)

In [ ]:
yp

In [ ]:
yp2

In [ ]:

xp = sorted(np.unique(sim_age))
yp = []
yp_s = []
for a in xp:
    r = (sim_rating[sim_age==a]*ovr_v).sum(1)
    yp.append(r.mean())
    yp_s.append(r.std())
yp = np.array(yp)
yp_s = np.array(yp_s)

xp2 = sorted(np.unique(sages_real))
yp2 = []
yp_s2 = []
for a in xp2:
    if (sages_real==a).sum() > 0:
        yp2.append(res_real[sages_real==a].mean())
        yp_s2.append(res_real[sages_real==a].std())
yp2 = np.array(yp2)
yp_s2 = np.array(yp_s2)

plt.style.use('fivethirtyeight')
plt.plot(xp2,yp2,label='rpd')
plt.fill_between(xp2,yp2+yp_s2,yp2-yp_s2,alpha=0.5)
plt.plot(xp,yp,label='sim')
plt.fill_between(xp,yp+yp_s,yp-yp_s,alpha=0.5)
plt.xlabel('age')
plt.ylabel('ovr')
plt.grid(True)
plt.xlim(20,37)
#plt.ylim(40,57)
plt.legend(frameon=True)

In [ ]:
#real_names[np.argsort(res_real)[-10:]]

In [ ]:
plt.hist((vec_real*ovr_v).sum(1),25,alpha=0.5,density=True)
plt.hist((sim_rating*ovr_v).sum(1),25,alpha=0.5,density=True)

In [ ]:
r_std2,v_std2,n_limits2

In [ ]:
mean_v2,p_model2

In [ ]:
for c,r in zip(cols, np.array([42, 43, 38, 29, 36, 35, 0, 40, 36, 37, 37, 45, 36, 49, 38])-mean_v2):
    print(c,r)


In [ ]:
plt.plot(np.arange(20,35),np.arange(20,35)*p_model2[0,0] +p_model2[0,1],label='new')
plt.plot(np.arange(20,35),np.arange(20,35)*base_model[0] +base_model[1],label='old')
plt.plot(np.arange(20,35),np.ones_like(np.arange(20,35))*p_model2[0,2],label='new')
plt.plot(np.arange(20,35),np.ones_like(np.arange(20,35))*base_model[2],label='old')

plt.plot([19,35],[0,0],c='k',ls='--')
plt.grid(True)
plt.legend()

In [ ]:
plt.style.use('seaborn-white')
for i in range(r1.shape[1]):
    plt.plot(np.arange(20,35),np.arange(20,35)*p_model2[1+i,0] +p_model2[1+i,1],label=cols[i],c=plt.cm.tab20(i))
plt.legend()
plt.xlim(19,35)
#plt.ylim(-4,4)
plt.grid(True)

In [ ]:
plt.figure(figsize=(8,6))
plt.style.use('seaborn-white')
for i in range(r1.shape[1]):
    plt.plot(np.arange(20,40),np.arange(20,40)*p_model2[0,0] +p_model2[0,1]+np.arange(20,40)*p_model2[1+i,0] +p_model2[1+i,1],label=cols[i],c=plt.cm.tab20(i))
plt.legend(loc='lower center', bbox_to_anchor=(0.5, 1.05),
          ncol=5, fancybox=True, shadow=True, borderaxespad=0.)
#plt.xlim(19,35)
#plt.ylim(-4,4)
plt.grid(True)
plt.tight_layout()

In [ ]:
for c, r in zip(cols,mean_v2):
    if c != 'hgt':
        print(c,':',int(np.round(r)),',')

In [ ]:
print(len(cols),len(p_model2))
for k,v in zip(['base'] + cols,p_model2):
    if k == 'hgt':continue
    print('{}: {},'.format(k,[round(_,3) for _ in v]))